In [1]:
from dotenv import load_dotenv
load_dotenv('.env')

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

input_text = "The meaning of life is 42"
vector = embeddings.embed_query(input_text)
vector

[-0.00056045176461339,
 -0.026505766436457634,
 0.007980902679264545,
 -0.0008843069663271308,
 -0.07557199895381927,
 0.0030145447235554457,
 0.019712621346116066,
 0.04048103094100952,
 -0.00907836388796568,
 -0.0013917958131060004,
 -0.014433695934712887,
 0.042759303003549576,
 0.015308885835111141,
 0.04395400732755661,
 0.014336451888084412,
 0.0354243740439415,
 -0.03472977876663208,
 0.021018462255597115,
 0.007286307401955128,
 -0.012245719321072102,
 -0.009203391149640083,
 0.004049492068588734,
 -0.04223141074180603,
 0.024019114673137665,
 -0.026339063420891762,
 0.024824844673275948,
 0.020726731047034264,
 -0.017100943252444267,
 -0.009606256149709225,
 0.020823974162340164,
 -0.0034174101892858744,
 0.017114834859967232,
 0.0701819360256195,
 -0.01739267259836197,
 0.005980467889457941,
 -0.04845498502254486,
 -0.004650317132472992,
 0.004570438992232084,
 -0.042175840586423874,
 0.03253485634922981,
 0.0004104625550098717,
 0.007828092202544212,
 -0.038036052137613297,


In [5]:
from qdrant_client import models, QdrantClient

client = QdrantClient(":memory:")

In [4]:
documents = [
    {
        "name": "The Time Machine",
        "description": "A man travels through time and witnesses the evolution of humanity.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game",
        "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Brave New World",
        "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Dune",
        "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Foundation",
        "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "The War of the Worlds",
        "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "The Hunger Games",
        "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "The Andromeda Strain",
        "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "The Three-Body Problem",
        "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]

In [6]:
client.create_collection(
    collection_name = "my_books",
    vectors_config = models.VectorParams(
        size = 3072, distance = models.Distance.COSINE
    )
)

True

In [8]:
points = [
    models.PointStruct(
        id=idx,
        vector=embeddings.embed_query(doc['description']),
        payload=doc
    ) for idx, doc in enumerate(documents)
]


ValueError: Collection my_bboks not found

In [9]:
client.upload_points(
    collection_name="my_books",
    points=points
)

In [10]:
hits = client.search(
    collection_name="my_books",
    query_vector=embeddings.embed_query("alien invasion"),
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'The War of the Worlds', 'description': 'A Martian invasion of Earth throws humanity into chaos.', 'author': 'H.G. Wells', 'year': 1898} score: 0.5440473142821519
{'name': 'The Three-Body Problem', 'description': 'Humans encounter an alien civilization that lives in a dying system.', 'author': 'Liu Cixin', 'year': 2008} score: 0.4777167664397118
{'name': 'The Andromeda Strain', 'description': 'A deadly virus from outer space threatens to wipe out humanity.', 'author': 'Michael Crichton', 'year': 1969} score: 0.47355968580267


In [12]:
import pandas as pd

pd.DataFrame([i.dict() for i in hits])

,id,version,score,payload,vector,shard_key,order_value
0,8,0,0.544047,"{'name': 'The War of the Worlds', 'description...",None,None,None
1,12,0,0.477717,"{'name': 'The Three-Body Problem', 'descriptio...",None,None,None
2,10,0,0.473560,"{'name': 'The Andromeda Strain', 'description'...",None,None,None
